In [1]:
import numpy as np

In [2]:
from pyomo.environ import *
from pyomo.environ import RangeSet
from pyomo.environ import value

In [3]:
A = np.array([
    [1,0,-1,0,0,0,0,0,0,0,0],
    [0,1,0,-1,0,0,0,0,0,0,0],
    [0,0,1,1,-1,0,0,0,0,0,0.6],
    [0,0,0,0,1,-1,0,0,0,0.5,0],
    [0,0,0,0,0,1,-1,0,0,0,0],
    [0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,-1,-1,0,0],
    [0,0,0,0,0,0,0,0,1,-1,-1]
])

In [4]:
np.shape(A)

(8, 11)

In [5]:
B = np.array ([
    [1,2,1,1,0,0,0,0.1,0.1,0.5,0.7]
])

In [6]:
C = np.array ([
    [1]
])

In [7]:
coef_GWP=C@B
coef_GWP=np.array(coef_GWP)
coef_GWP = coef_GWP.reshape(-1)

In [8]:
F=1
f=np.zeros(len(A))
f[5]=F

In [9]:
len(A)

8

In [10]:
positive_s_indices = []
positive_s_indices = list(range(1, len(np.transpose(A))+1))
#Model Formulation
# Create the model
model = ConcreteModel()

# Define the decision variable
model.set_s = RangeSet(len(np.transpose(A)))
model.s = Var(model.set_s)
model.set_positive_scale = Set(initialize=positive_s_indices)

#model constraint: As = f and s>=0
model.set_balance = RangeSet(len(f))
def balance(model, p): # As = f
    return sum(A[p-1,i-1]*model.s[i] for i in model.set_s) == f[p-1]
def positive_scale(model, i): # s>=0
    return (model.s[i]>=0)

model.balance_constraints = Constraint(model.set_balance, rule=balance)
model.positive_scale_constraints = Constraint(model.set_positive_scale, rule=positive_scale)

model.obj = Objective(expr = sum(coef_GWP[i-1]*model.s[i] for i in model.set_s), sense=minimize)
#Solver
solver = SolverFactory('gurobi')
solver.solve(model) # solves and updates instance

{'Problem': [{'Name': 'x12', 'Lower bound': 1.6, 'Upper bound': 1.6, 'Number of objectives': 1, 'Number of constraints': 20, 'Number of variables': 12, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 12, 'Number of nonzeros': 32, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.0006899833679199219', 'Error rc': 0, 'Time': 0.07832098007202148}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [11]:
value(model.obj)

1.6

In [12]:
scaling_factors = []
for j in model.s:
    scaling_factors.append(model.s[j].value)       

In [13]:
scaling_factors

[0.4, 0.0, 0.4, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0]